In [1]:
import pandas as pd

data = pd.read_csv('ChnSentiCorp_htl_all.csv')
data.head()

,label,review
0,1,"距離川沙公路較近,但是公交指示不對,如果是""蔡陸線""的話,會非常麻煩.建議用別的路線.房間較..."
1,1,商務大床房，房間很大，床有2M寬，整體感覺經濟實惠不錯!
2,1,早餐太差，無論去多少人，那邊也不加食品的。酒店應該重視一下這個問題了。房間本身很好。
3,1,賓館在小街道上，不大好找，但還好北京熱心同胞很多~賓館設施跟介紹的差不多，房間很小，確實挺小...
4,1,"CBD中心,周圍沒什麼店鋪,說5星有點勉強.不知道為什麼衛生間沒有電吹風"


In [4]:
data = data.dropna()
len(data)

7765

In [ ]:
from torch.utils.data import Dataset

class HotelDataSet(Dataset):
    def __init__(self):
        super().__init__()
        data = pd.read_csv('ChnSentiCorp_htl_all.csv')
        self.data = data.dropna()

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index): #傳出tuple
        return self.data.iloc[index]['review'], self.data.iloc[index]['label']

hotelDataSet = HotelDataSet()
hotelDataSet[0]

('距離川沙公路較近,但是公交指示不對,如果是"蔡陸線"的話,會非常麻煩.建議用別的路線.房間較為簡單.', np.int64(1))